In [1]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from cv_vis import plot_training_stats

***AlexNet***

This is a simple analysis of general features of a simplified implementation (see NNs/AlexNet.py and runtime parameters in globals.py and cv.py) of the image recognition network AlexNet (Krizhevsky, Sutskever, and Hinton 2012).

AlexNet is a convolutional neural network with four max-pooled convolutional layers followed by two dense layers. For original structure, see Fig. 2 of the original paper (reproduced here). It is best known for being the first deep neural network to outperform traditional machine learning tools in the ImageNet challenge (image-net.org), in 2012.